In [3]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import random
import time
import os



def extract_person_urls(driver):
    person_links = driver.find_elements(By.CSS_SELECTOR, "a.nombre")
    
    urls = []
    for link in person_links:
        url = link.get_attribute("href")
        if url:
            urls.append(url)
    
    return urls

def random_delay():
    delay = random.uniform(1, 3)
    time.sleep(delay)
    return delay

def scrape_all_pages(driver):
    all_urls = []
    page_number = 0
    
    while True:
        try:
            #delay = random_delay()
            #print(f"Esperando {delay:.2f} segundos antes de procesar la página {page_number}")
            
            # Extraer URLs de la página actual
            page_urls = extract_person_urls(driver)
            all_urls.extend(page_urls)
            print(f"Página {page_number}: Encontradas {len(page_urls)} URLs")
            
            # Buscar el botón de "Página Siguiente"
            try:
                next_button = driver.find_element(By.XPATH, "//a[contains(text(), 'Página Siguiente')]")
            except NoSuchElementException:
                print("No se encontró el botón de siguiente página")
                break
            
            # Verificar si el botón está deshabilitado
            parent_li = next_button.find_element(By.XPATH, "./..")
            if 'disabled' in parent_li.get_attribute('class'):
                print("Llegamos a la última página")
                break
                
            # Obtener la URL de la siguiente página
            next_url = next_button.get_attribute('href')
            if not next_url:
                print("No hay más páginas disponibles")
                break
                
            # Navegar a la siguiente página
            driver.get(next_url)
            page_number += 1
            
        except Exception as e:
            print(f"Error inesperado: {str(e)}")
            break
    
    return all_urls


## Docker

In [7]:

def create_driver():
    chrome_options = Options()
    
    # Basic configurations for running in container
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    
    # Get selenium host and port from environment variables
    selenium_host = os.getenv('SELENIUM_HOST', 'selenium-hub')
    selenium_port = os.getenv('SELENIUM_PORT', '4444')
    
    # Set up remote webdriver
    try:
        driver = webdriver.Remote(
            command_executor=f'http://{selenium_host}:{selenium_port}/wd/hub',
            options=chrome_options
        )
        
        return driver
    
    except Exception as e:
        print(f"Error creating driver: {str(e)}")
        raise


driver = None
try:
    # Create driver
    driver = create_driver()
    
    # Navigate to the website
    driver.get("https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa")
    
    # Extraer URLs de todas las páginas
    all_person_urls = scrape_all_pages(driver)
    
    # Imprimir resultados
    print(f"\nResultados finales:")
    print(f"Total de URLs encontradas: {len(all_person_urls)}")
    for url in all_person_urls:
        print(url)
        
except Exception as e:
    print(f"Error durante el scraping: {str(e)}")
    
finally:
    if driver:
        try:
            driver.quit()
        except Exception as e:
            print(f"Error closing driver: {str(e)}")


Página 0: Encontradas 12 URLs
Página 1: Encontradas 12 URLs
Página 2: Encontradas 12 URLs
Página 3: Encontradas 12 URLs
Página 4: Encontradas 12 URLs
Página 5: Encontradas 12 URLs
Página 6: Encontradas 12 URLs
Página 7: Encontradas 12 URLs
Página 8: Encontradas 12 URLs
Página 9: Encontradas 12 URLs
Página 10: Encontradas 12 URLs
Página 11: Encontradas 12 URLs
Página 12: Encontradas 12 URLs
Página 13: Encontradas 12 URLs
Página 14: Encontradas 12 URLs
Página 15: Encontradas 12 URLs
Página 16: Encontradas 12 URLs
Página 17: Encontradas 12 URLs
Página 18: Encontradas 12 URLs
Página 19: Encontradas 12 URLs
Página 20: Encontradas 12 URLs
Página 21: Encontradas 12 URLs
Página 22: Encontradas 12 URLs
Página 23: Encontradas 12 URLs
Página 24: Encontradas 12 URLs
Página 25: Encontradas 12 URLs
Página 26: Encontradas 12 URLs
Página 27: Encontradas 12 URLs
Página 28: Encontradas 12 URLs
Página 29: Encontradas 12 URLs
Página 30: Encontradas 12 URLs
Página 31: Encontradas 12 URLs
Página 32: Encontr

In [12]:
df = pd.DataFrame(all_person_urls, columns=['URL'])
df.to_csv('data/chiapas/urls.csv', index=False)

---
---

## Local

In [ ]:
chrome_options = Options()

# Basic configurations
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')

driver = webdriver.Chrome(options=chrome_options)
driver.get("https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa")

try:
    # Extraer URLs de todas las páginas
    all_person_urls = scrape_all_pages(driver)
    
    # Imprimir resultados
    print(f"\nResultados finales:")
    print(f"Total de URLs encontradas: {len(all_person_urls)}")
    for url in all_person_urls:
        print(url)
        
except Exception as e:
    print(f"Error durante el scraping: {str(e)}")
finally:
    # No cerramos el driver aquí ya que fue pasado como parámetro
    pass

In [ ]:
df = pd.DataFrame(all_person_urls, columns=['URL'])
df.to_csv('../../../data/chiapas/urls.csv', index=False)

---
---

In [5]:
df = pd.read_csv('data/chiapas/urls.csv')
all_person_urls = df['URL'].tolist()
all_person_urls

['https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/2E10FDC1-3287-4304-AA94-988962331061',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/8191B3BC-45E9-40ED-962F-B5B7B7ADEAFE',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/58EF1754-276E-438C-BA94-69EA2EF20DA4',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/AF5AEDD1-0C73-4BF9-9D01-E1AC49DF5312',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/5D07EE8A-B688-4E2E-8C66-EDD5036EFF37',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/16B25E97-7815-48D6-9D72-6F037F5154ED',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/A63CDA72-9AC4-4D9B-AFA8-E7DFF36C9ED4',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/82074C20-7424-4754-BE8E-498FA2BBAC3F',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/E070E7DB-BBEA-46FF-B255-6228A1F023FC',
 'https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/49929353-075A-470C-

---
---
## Local

In [ ]:
import cv2
import numpy as np
import pytesseract
import requests
from PIL import Image
import re
from datetime import datetime
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urljoin

class PersonAnalyzer:
    def __init__(self, driver_path=None, headless=True):
        """
        Inicializa el analizador con opciones de Selenium
        """
        self.setup_selenium(driver_path, headless)
        
    def setup_selenium(self, driver_path, headless):
        """Configura el driver de Selenium"""
        options = webdriver.ChromeOptions()
        if headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        
        if driver_path:
            self.driver = webdriver.Chrome(driver_path, options=options)
        else:
            self.driver = webdriver.Chrome(options=options)
        
        self.driver.set_page_load_timeout(30)
        self.wait = WebDriverWait(self.driver, 10)

    def clean_text(self, text):
        """Limpia y normaliza el texto"""
        if not text:
            return ""
        text = re.sub(r'\s+', ' ', text.strip())
        return text.upper()

    def parse_date(self, date_str):
        """Convierte string de fecha a formato estándar"""
        try:
            date = datetime.strptime(date_str.strip(), '%d/%m/%Y')
            return date.strftime('%Y-%m-%d')
        except:
            return date_str

    def safe_get_text(self, xpath, default=""):
        """Obtiene texto de manera segura con manejo de errores y espera"""
        try:
            element = self.wait.until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            return self.clean_text(element.text)
        except:
            return default

    def extract_selenium_data(self, url, analizar_imagen=True):
        """Extrae información usando selectores de Selenium"""
        data = {}
        try:
            # Acceder a la URL principal
            base_url = "https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/"
            guid = url.split('/')[-1]
            self.driver.get(f"{base_url}{guid}")
            
            # Esperar a que cargue el contenido principal
            self.wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, "emp-profile"))
            )
            
            # Extraer URL de la imagen
            try:
                img_element = self.driver.find_element(By.XPATH, "//div[@class='profile-img']/img")
                data['imagen_url'] = img_element.get_attribute('src')
            except:
                data['imagen_url'] = None

            # Información básica
            data['nombre'] = self.safe_get_text("//div[@class='profile-head']//h3")
            data['registro'] = self.safe_get_text("//p[@class='proile-rating']/span")
            data['url'] = url
            
            # Campos principales
            fields = {
                'SEXO': "//label[contains(text(), 'Sexo:')]/../..//p",
                'ESTATURA': "//label[contains(text(), 'Estatura:')]/../..//p",
                'TEZ': "//label[contains(text(), 'Tez:')]/../..//p",
                'OJOS': "//label[contains(text(), 'Ojos:')]/../..//p",
                'CABELLO': "//label[contains(text(), 'Cabello:')]/../..//p",
                'PESO': "//label[contains(text(), 'Peso:')]/../..//p",
                'FECHA_DESAPARICION': "//label[contains(text(), 'Fecha desaparición:')]/../..//p",
                'COMPLEXION': "//label[contains(text(), 'Complexion:')]/../..//p",
                'BOCA': "//label[contains(text(), 'Boca:')]/../..//p",
                'TAMAÑO_NARIZ': "//label[contains(text(), 'Tamaño de nariz')]/../..//p",
                'TIPO_NARIZ': "//label[contains(text(), 'Tipo de nariz:')]/../..//p",
                'ESCOLARIDAD': "//label[contains(text(), 'Escolaridad:')]/../..//p",
                'ORIGINARIO': "//label[contains(text(), 'Originario de:')]/../..//p"
            }
            
            for field, xpath in fields.items():
                data[field] = self.safe_get_text(xpath)

            # Información adicional
            data['FECHA_NACIMIENTO'] = self.parse_date(
                self.safe_get_text("//b[contains(text(), 'Fecha de nacimiento:')]/following-sibling::p")
            )
            
            data['SEÑAS_PARTICULARES'] = self.safe_get_text(
                "//strong[contains(text(), 'Señas Particulares:')]/following-sibling::p"
            )
            
            data['CIRCUNSTANCIA'] = self.safe_get_text(
                "//strong[contains(text(), 'Circunstancia:')]/following-sibling::p"
            )

            # Analizar imagen si está disponible
            if data.get('imagen_url') and analizar_imagen:
                is_located, detected_text = self.analyze_image(data['imagen_url'])
                data['is_located'] = is_located
                data['detected_text'] = detected_text
            
            return data
            
        except Exception as e:
            raise Exception(f"Error extrayendo datos: {str(e)}")

    def analyze_image(self, url):
        """Analiza la imagen para detectar si está localizada"""
        try:
            # Obtener la imagen
            response = requests.get(url)
            img_array = np.frombuffer(response.content, np.uint8)
            image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
            
            if image is None:
                raise Exception("No se pudo cargar la imagen")
            
            # Redimensionar la imagen si es muy grande
            height, width = image.shape[:2]
            max_dimension = 1200
            if max(height, width) > max_dimension:
                scale = max_dimension / max(height, width)
                image = cv2.resize(image, None, fx=scale, fy=scale)
            
    # Convertir a HSV para detectar rojo
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            
            # Definir rangos de color rojo en HSV
            lower_red1 = np.array([0, 100, 100])
            upper_red1 = np.array([10, 255, 255])
            lower_red2 = np.array([160, 100, 100])
            upper_red2 = np.array([180, 255, 255])
            
            # Crear máscara para texto rojo
            mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
            mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
            mask = cv2.bitwise_or(mask1, mask2)
            
            # Aplicar máscara
            red_text = cv2.bitwise_and(image, image, mask=mask)
            
            # Preparar imagen para OCR
            gray = cv2.cvtColor(red_text, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Convertir la imagen de OpenCV a formato PIL
            pil_image = Image.fromarray(binary)
            
            # Realizar OCR usando la imagen PIL
            try:
                text = pytesseract.image_to_string(pil_image, lang='spa')
            except:
                print("Fallback a idioma inglés...")
                text = pytesseract.image_to_string(pil_image)
            
            # Limpiar y normalizar el texto detectado
            text = ' '.join(text.split())
            text = text.upper()
            
            # Lista ampliada de frases a buscar
            localization_phrases = [
                'PERSONA LOCALIZADA',
                'LOCALIZADA',
                'LOCALIZAD',
                'LOCALIZ',
                'LOCAL',
                'PERS'
            ]
            
            is_located = any(phrase in text for phrase in localization_phrases)
            
            return is_located, text
            
        except Exception as e:
            print(f"Error en análisis de imagen: {str(e)}")
            return False, ""

    def analyze_person_url(self, url, analizar_imagen):
        """Analiza una persona a partir de su URL"""
        try:
            # Extraer datos usando Selenium
            data = self.extract_selenium_data(url, analizar_imagen)
            
            return {
                'success': True,
                'data': data
            }
            
        except Exception as e:
            return {
                'success': False,
                'url': url,
                'error': str(e)
            }

    def analyze_multiple_urls(self, urls, output_file=None, analizar_imagen=True):
        """Analiza múltiples URLs y opcionalmente guarda los resultados en un archivo"""
        results = []
        total = len(urls)
        
        for i, url in enumerate(urls, 1):
            print(f"\nProcesando URL {i}/{total}: {url}")
            try:
                # Agregar reintento en caso de error
                max_retries = 3
                retry_count = 0
                while retry_count < max_retries:
                    try:
                        result = self.analyze_person_url(url, analizar_imagen)
                        break
                    except Exception as e:
                        retry_count += 1
                        if retry_count == max_retries:
                            result = {
                                'success': False,
                                'url': url,
                                'error': str(e)
                            }
                        else:
                            print(f"Reintento {retry_count} para URL {url}")
                            time.sleep(2)  # Esperar antes de reintentar
                
                results.append(result)
                
                # Guardar resultados parciales
                if output_file and result['success']:
                    df = pd.DataFrame([result['data']])
                    mode = 'a' if i > 1 else 'w'
                    header = i == 1
                    df.to_csv(output_file, mode=mode, header=header, index=False)
            
            except Exception as e:
                print(f"Error procesando URL {url}: {str(e)}")
                results.append({
                    'success': False,
                    'url': url,
                    'error': str(e)
                })
        
        return results

    def close(self):
        """Cierra el driver de Selenium"""
        if hasattr(self, 'driver'):
            self.driver.quit()

def print_person_info(result):
    """Imprime la información de la persona de manera formateada"""
    if result['success']:
        data = result['data']
        print("\n=== INFORMACIÓN DE LA PERSONA ===")
        print(f"Nombre: {data.get('nombre', 'No disponible')}")
        print(f"Registro: {data.get('registro', 'No disponible')}")
        print(f"URL: {data.get('url', 'No disponible')}")
        
        print(f"\n--- CARACTERÍSTICAS FÍSICAS ---")
        print(f"Sexo: {data.get('SEXO', 'No disponible')}")
        print(f"Estatura: {data.get('ESTATURA', 'No disponible')}")
        print(f"Peso: {data.get('PESO', 'No disponible')}")
        print(f"Tez: {data.get('TEZ', 'No disponible')}")
        print(f"Complexión: {data.get('COMPLEXION', 'No disponible')}")
        print(f"Ojos: {data.get('OJOS', 'No disponible')}")
        print(f"Cabello: {data.get('CABELLO', 'No disponible')}")
        print(f"Boca: {data.get('BOCA', 'No disponible')}")
        print(f"Nariz: {data.get('TAMAÑO_NARIZ', 'No disponible')} - {data.get('TIPO_NARIZ', 'No disponible')}")
        
        print(f"\n--- INFORMACIÓN PERSONAL ---")
        print(f"Fecha de nacimiento: {data.get('FECHA_NACIMIENTO', 'No disponible')}")
        print(f"Escolaridad: {data.get('ESCOLARIDAD', 'No disponible')}")
        print(f"Originario de: {data.get('ORIGINARIO', 'No disponible')}")
        
        print(f"\n--- INFORMACIÓN DE DESAPARICIÓN ---")
        print(f"Fecha de desaparición: {data.get('FECHA_DESAPARICION', 'No disponible')}")
        print(f"Señas particulares: {data.get('SEÑAS_PARTICULARES', 'No disponible')}")
        print(f"Circunstancia: {data.get('CIRCUNSTANCIA', 'No disponible')}")
        
        print(f"\n--- ESTADO ---")
        print(f"¿Persona localizada?: {'Sí' if data.get('is_located', False) else 'No'}")
        if 'detected_text' in data:
            print(f"Texto detectado en imagen: {data['detected_text']}")
    else:
        print(f"\nError procesando información: {result['error']}")

In [ ]:
# Crear el analizador
analyzer = PersonAnalyzer(headless=True)  # headless=True para ejecutar Chrome sin interfaz gráfica

try:

# Procesar todas las URLs y guardar en CSV
   results = analyzer.analyze_multiple_urls(all_person_urls, output_file="data/chiapas/personas.csv", analizar_imagen=False)
   
   # Imprimir resultados
   for result in results:
       print_person_info(result)

finally:
   # Asegurarse de cerrar el navegador
   analyzer.close()

---
---
## Docker

In [7]:
import cv2
import numpy as np
import pytesseract
import requests
from PIL import Image
import re
from datetime import datetime
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from urllib.parse import urljoin

class PersonAnalyzer:
    def __init__(self, headless=True):
        """
        Inicializa el analizador con opciones de Selenium para ambiente dockerizado
        """
        self.setup_selenium(headless)
        
    def setup_selenium(self, headless):
        """Configura el driver de Selenium para conexión remota"""
        options = Options()
        
        # Configuraciones básicas para ambiente containerizado
        if headless:
            options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-dev-shm-usage')
        options.add_argument('--disable-gpu')
        options.add_argument('--window-size=1920,1080')
        
        # Obtener host y puerto de variables de ambiente
        selenium_host = os.getenv('SELENIUM_HOST', 'selenium-hub')
        selenium_port = os.getenv('SELENIUM_PORT', '4444')
        
        # Configurar conexión remota
        try:
            self.driver = webdriver.Remote(
                command_executor=f'http://{selenium_host}:{selenium_port}/wd/hub',
                options=options
            )
            self.driver.set_page_load_timeout(30)
            self.wait = WebDriverWait(self.driver, 10)
            print(f"Conexión exitosa al hub de Selenium en {selenium_host}:{selenium_port}")
        except Exception as e:
            raise Exception(f"Error conectando al hub de Selenium: {str(e)}")

    def clean_text(self, text):
            """Limpia y normaliza el texto"""
            if not text:
                return ""
            text = re.sub(r'\s+', ' ', text.strip())
            return text.upper()

    def parse_date(self, date_str):
        """Convierte string de fecha a formato estándar"""
        try:
            date = datetime.strptime(date_str.strip(), '%d/%m/%Y')
            return date.strftime('%Y-%m-%d')
        except:
            return date_str

    def safe_get_text(self, xpath, default=""):
        """Obtiene texto de manera segura con manejo de errores y espera"""
        try:
            element = self.wait.until(
                EC.presence_of_element_located((By.XPATH, xpath))
            )
            return self.clean_text(element.text)
        except:
            return default

    def extract_selenium_data(self, url, analizar_imagen=True):
        """Extrae información usando selectores de Selenium"""
        data = {}
        try:
            # Acceder a la URL principal
            base_url = "https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/"
            guid = url.split('/')[-1]
            self.driver.get(f"{base_url}{guid}")
            
            # Esperar a que cargue el contenido principal
            self.wait.until(
                EC.presence_of_element_located((By.CLASS_NAME, "emp-profile"))
            )
            
            # Extraer URL de la imagen
            try:
                img_element = self.driver.find_element(By.XPATH, "//div[@class='profile-img']/img")
                data['imagen_url'] = img_element.get_attribute('src')
            except:
                data['imagen_url'] = None

            # Información básica
            data['nombre'] = self.safe_get_text("//div[@class='profile-head']//h3")
            data['registro'] = self.safe_get_text("//p[@class='proile-rating']/span")
            data['url'] = url
            
            # Campos principales
            fields = {
                'SEXO': "//label[contains(text(), 'Sexo:')]/../..//p",
                'ESTATURA': "//label[contains(text(), 'Estatura:')]/../..//p",
                'TEZ': "//label[contains(text(), 'Tez:')]/../..//p",
                'OJOS': "//label[contains(text(), 'Ojos:')]/../..//p",
                'CABELLO': "//label[contains(text(), 'Cabello:')]/../..//p",
                'PESO': "//label[contains(text(), 'Peso:')]/../..//p",
                'FECHA_DESAPARICION': "//label[contains(text(), 'Fecha desaparición:')]/../..//p",
                'COMPLEXION': "//label[contains(text(), 'Complexion:')]/../..//p",
                'BOCA': "//label[contains(text(), 'Boca:')]/../..//p",
                'TAMAÑO_NARIZ': "//label[contains(text(), 'Tamaño de nariz')]/../..//p",
                'TIPO_NARIZ': "//label[contains(text(), 'Tipo de nariz:')]/../..//p",
                'ESCOLARIDAD': "//label[contains(text(), 'Escolaridad:')]/../..//p",
                'ORIGINARIO': "//label[contains(text(), 'Originario de:')]/../..//p"
            }
            
            for field, xpath in fields.items():
                data[field] = self.safe_get_text(xpath)

            # Información adicional
            data['FECHA_NACIMIENTO'] = self.parse_date(
                self.safe_get_text("//b[contains(text(), 'Fecha de nacimiento:')]/following-sibling::p")
            )
            
            data['SEÑAS_PARTICULARES'] = self.safe_get_text(
                "//strong[contains(text(), 'Señas Particulares:')]/following-sibling::p"
            )
            
            data['CIRCUNSTANCIA'] = self.safe_get_text(
                "//strong[contains(text(), 'Circunstancia:')]/following-sibling::p"
            )

            # Analizar imagen si está disponible
            if data.get('imagen_url') and analizar_imagen:
                is_located, detected_text = self.analyze_image(data['imagen_url'])
                data['is_located'] = is_located
                data['detected_text'] = detected_text
            
            return data
            
        except Exception as e:
            raise Exception(f"Error extrayendo datos: {str(e)}")

    def analyze_image(self, url):
        """Analiza la imagen para detectar si está localizada"""
        try:
            # Obtener la imagen
            response = requests.get(url)
            img_array = np.frombuffer(response.content, np.uint8)
            image = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
            
            if image is None:
                raise Exception("No se pudo cargar la imagen")
            
            # Redimensionar la imagen si es muy grande
            height, width = image.shape[:2]
            max_dimension = 1200
            if max(height, width) > max_dimension:
                scale = max_dimension / max(height, width)
                image = cv2.resize(image, None, fx=scale, fy=scale)
            
    # Convertir a HSV para detectar rojo
            hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
            
            # Definir rangos de color rojo en HSV
            lower_red1 = np.array([0, 100, 100])
            upper_red1 = np.array([10, 255, 255])
            lower_red2 = np.array([160, 100, 100])
            upper_red2 = np.array([180, 255, 255])
            
            # Crear máscara para texto rojo
            mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
            mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
            mask = cv2.bitwise_or(mask1, mask2)
            
            # Aplicar máscara
            red_text = cv2.bitwise_and(image, image, mask=mask)
            
            # Preparar imagen para OCR
            gray = cv2.cvtColor(red_text, cv2.COLOR_BGR2GRAY)
            _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Convertir la imagen de OpenCV a formato PIL
            pil_image = Image.fromarray(binary)
            
            # Realizar OCR usando la imagen PIL
            try:
                text = pytesseract.image_to_string(pil_image, lang='spa')
            except:
                print("Fallback a idioma inglés...")
                text = pytesseract.image_to_string(pil_image)
            
            # Limpiar y normalizar el texto detectado
            text = ' '.join(text.split())
            text = text.upper()
            
            # Lista ampliada de frases a buscar
            localization_phrases = [
                'PERSONA LOCALIZADA',
                'LOCALIZADA',
                'LOCALIZAD',
                'LOCALIZ',
                'LOCAL',
                'PERS'
            ]
            
            is_located = any(phrase in text for phrase in localization_phrases)
            
            return is_located, text
            
        except Exception as e:
            print(f"Error en análisis de imagen: {str(e)}")
            return False, ""

    def analyze_person_url(self, url, analizar_imagen):
        """Analiza una persona a partir de su URL"""
        try:
            # Extraer datos usando Selenium
            data = self.extract_selenium_data(url, analizar_imagen)
            
            return {
                'success': True,
                'data': data
            }
            
        except Exception as e:
            return {
                'success': False,
                'url': url,
                'error': str(e)
            }
            
    def analyze_multiple_urls(self, urls, output_file=None, analizar_imagen=True):
        """Analiza múltiples URLs y opcionalmente guarda los resultados en un archivo"""
        results = []
        total = len(urls)
        
        # Asegurarse que el directorio de salida existe
        if output_file:
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
        
        for i, url in enumerate(urls, 1):
            print(f"\nProcesando URL {i}/{total}: {url}")
            try:
                # Agregar reintento en caso de error
                max_retries = 3
                retry_count = 0
                while retry_count < max_retries:
                    try:
                        result = self.analyze_person_url(url, analizar_imagen)
                        break
                    except Exception as e:
                        retry_count += 1
                        if retry_count == max_retries:
                            result = {
                                'success': False,
                                'url': url,
                                'error': str(e)
                            }
                        else:
                            print(f"Reintento {retry_count} para URL {url}")
                            # Reconectar al hub si es necesario
                            try:
                                self.driver.quit()
                                self.setup_selenium(headless=True)
                            except:
                                pass
                            time.sleep(2)  # Esperar antes de reintentar
                
                results.append(result)
                
                # Guardar resultados parciales
                if output_file and result['success']:
                    df = pd.DataFrame([result['data']])
                    mode = 'a' if i > 1 else 'w'
                    header = i == 1
                    df.to_csv(output_file, mode=mode, header=header, index=False)
            
            except Exception as e:
                print(f"Error procesando URL {url}: {str(e)}")
                results.append({
                    'success': False,
                    'url': url,
                    'error': str(e)
                })
        
        return results
    
    def close(self):
        """Cierra el driver de Selenium"""
        if hasattr(self, 'driver'):
            self.driver.quit()

In [8]:
# Crear el analizador
analyzer = PersonAnalyzer(headless=True)  # headless=True para ejecutar Chrome sin interfaz gráfica
try:
# Procesar todas las URLs y guardar en CSV
   results = analyzer.analyze_multiple_urls(all_person_urls, output_file="data/chiapas/personas.csv", analizar_imagen=False)
   

finally:
   # Asegurarse de cerrar el navegador
   analyzer.close()

Conexión exitosa al hub de Selenium en selenium-hub:4444

Procesando URL 1/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/2E10FDC1-3287-4304-AA94-988962331061

Procesando URL 2/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/8191B3BC-45E9-40ED-962F-B5B7B7ADEAFE

Procesando URL 3/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/58EF1754-276E-438C-BA94-69EA2EF20DA4

Procesando URL 4/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/AF5AEDD1-0C73-4BF9-9D01-E1AC49DF5312

Procesando URL 5/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/5D07EE8A-B688-4E2E-8C66-EDD5036EFF37

Procesando URL 6/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/16B25E97-7815-48D6-9D72-6F037F5154ED

Procesando URL 7/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/A63CDA72-9AC4-4D9B-AFA8-E7DFF36C9ED4

Procesando URL 8/1135: https://www.fge.chiapas.gob.mx/Servicios/Hasvistoa/HASVISTOA/8207

NameError: name 'print_person_info' is not defined